In [126]:
from elasticsearch import Elasticsearch
from datetime import datetime
from flask import Flask, render_template, request
import csv
import re


In [127]:

es = Elasticsearch([{'host': 'localhost', 'port':9200, 'scheme': 'http'}], http_auth=("vijay", "password"))
es.info()

C:\Users\vijay\AppData\Local\Temp\ipykernel_36848\2575928368.py:1: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch([{'host': 'localhost', 'port':9200, 'scheme': 'http'}], http_auth=("vijay", "password"))


ObjectApiResponse({'name': 'BOOK-GM09OBS762', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'vJxnvQteTUeURso4lvYqKw', 'version': {'number': '8.10.4', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': 'b4a62ac808e886ff032700c391f45f1408b2538c', 'build_date': '2023-10-11T22:04:35.506990650Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [128]:
cnbc_documents = list()
guardian_documents = list()
reuters_documents = list()

with open("cnbc_headlines.csv", "r", encoding='utf-8') as file:

    reader = csv.reader(file)

    headers = next(reader)

    for row in reader:

        data = dict()

        data['News Organization'] = 'CNBC'

        i = 0

        for h in headers:

            data[h] = row[i]
            i += 1
        
        cnbc_documents.append(data)


with open("guardian_headlines.csv", "r", encoding='utf-8') as file:

    reader = csv.reader(file)

    headers = next(reader)

    for row in reader:

        data = dict()

        data['News Organization'] = 'The Guardian'

        i = 0

        for h in headers:

            data[h] = row[i]
            i += 1
        
        guardian_documents.append(data)


with open("reuters_headlines.csv", "r", encoding='utf-8') as file:

    reader = csv.reader(file)

    headers = next(reader)

    for row in reader:

        data = dict()

        data['News Organization'] = 'Reuters'

        i = 0

        for h in headers:

            data[h] = row[i]
            i += 1
        
        reuters_documents.append(data)


print(cnbc_documents[:10])
print()

print(guardian_documents[:10])
print()

print(reuters_documents[:10])
print()

[{'News Organization': 'CNBC', 'Headlines': 'Jim Cramer: A better way to invest in the Covid-19 vaccine gold rush', 'Time': ' 7:51  PM ET Fri, 17 July 2020', 'Description': '"Mad Money" host Jim Cramer recommended buying four companies that are supporting vaccine developers.'}, {'News Organization': 'CNBC', 'Headlines': "Cramer's lightning round: I would own Teradyne", 'Time': ' 7:33  PM ET Fri, 17 July 2020', 'Description': '"Mad Money" host Jim Cramer rings the lightning round bell, which means he\'s giving his answers to callers\' stock questions at rapid speed.'}, {'News Organization': 'CNBC', 'Headlines': '', 'Time': '', 'Description': ''}, {'News Organization': 'CNBC', 'Headlines': "Cramer's week ahead: Big week for earnings, even bigger week for vaccines", 'Time': ' 7:25  PM ET Fri, 17 July 2020', 'Description': '"We\'ll pay more for the earnings of the non-Covid companies if The Lancet publishes some good news from AstraZeneca\'s vaccine trial," Cramer said.'}, {'News Organizat

In [129]:
def index_documents(index, id, document):
    
    resp = es.index(index=index, id=id, document=document)


In [130]:
for i in range(len(cnbc_documents)):
    
    es.index(index = "cnbc_index", id = i + 1, document = cnbc_documents[i])

for i in range(len(guardian_documents)):
    
    es.index(index = "guardian_index", id = i + 1, document = guardian_documents[i])

for i in range(len(reuters_documents)):
    
    es.index(index = "reuters_index", id = i + 1, document = reuters_documents[i])

In [131]:
resp = es.get(index="cnbc_index", id=1)
print(resp['_source'])
print()

resp = es.get(index="guardian_index", id=1)
print(resp['_source'])
print()

resp = es.get(index="reuters_index", id=1)
print(resp['_source'])
print()

{'News Organization': 'CNBC', 'Headlines': 'Jim Cramer: A better way to invest in the Covid-19 vaccine gold rush', 'Time': ' 7:51  PM ET Fri, 17 July 2020', 'Description': '"Mad Money" host Jim Cramer recommended buying four companies that are supporting vaccine developers.'}

{'News Organization': 'The Guardian', 'Time': '18-Jul-20', 'Headlines': 'Johnson is asking Santa for a Christmas recovery'}

{'News Organization': 'Reuters', 'Headlines': 'TikTok considers London and other locations for headquarters', 'Time': 'Jul 18 2020', 'Description': 'TikTok has been in discussions with the UK government over the past few months to locate its headquarters in London, a source familiar with the matter said, as part of a strategy to distance itself from its Chinese ownership.'}



In [132]:

def search_query(query, datasets_to_search, index_names):
    
    body = {
        "size": 20,
        "query": {
            "dis_max": {
                "queries": [
                    {"match": {"Headlines": query}},
                    {"match": {"Description": query}}
                ],
                "tie_breaker": 0.3
            }
        }
    }

    results = []

    for i in range(len(datasets_to_search)):

        resp = es.search(index=index_names[i], body=body)

        for hit in resp['hits']['hits']:
            
            news_organization = hit["_source"].get('News Organization')
            headline = hit["_source"].get('Headlines')
            time = hit["_source"].get('Time')
            description = hit["_source"].get('Description')

            formatted_result = f"'News Organization': {news_organization}\nHeadline: {headline}\nTime: {time}\nDescription: {description}\n\n"
            
            formatted_result = formatted_result.replace('\n', '<br>')

            results.append(formatted_result)

    return results


In [133]:
app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def homepage():

    results = []

    if request.method == 'POST':

        query = request.form.get('search_query')

        datasets_to_search = list()
        index_names = list()

        if request.form.get('cnbc'):

            datasets_to_search.append(cnbc_documents)
            index_names.append("cnbc_index")
        
        if request.form.get('the guardian'):

            datasets_to_search.append(guardian_documents)
            index_names.append("guardian_index")
        
        if request.form.get('reuters'):

            datasets_to_search.append(reuters_documents)
            index_names.append("reuters_index")

        results = search(query, datasets_to_search, index_names)
    
    return render_template('test.html', results=results)

def search(query, datasets_to_search, index_names):

    return search_query(query, datasets_to_search, index_names)


app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [30/Nov/2023 09:11:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2023 09:11:32] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [30/Nov/2023 09:11:41] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2023 09:12:08] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2023 09:12:51] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2023 09:13:39] "POST / HTTP/1.1" 200 -
